In [1]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import time
import json
import sqlite3
from datetime import datetime
import re
import datetime

In [2]:
def filter(item):
    item = item.replace("’","'")
    item = item.replace("”",'"')
    item = item.replace("“",'"')
    item = item.replace("…",' ')
    item = item.replace("&amp;",' and ')
    item = re.sub("([…:;|])", " ", item)
    item = item.replace("—",'-')
    item = item.replace("\n",' ')
    item = item.replace("\t",' ')
    #item = item.replace("([a-z])\\1+", r"\1")
    item = re.sub("([a-zA-Z])\\1+\\1+", r"\1", item)
    item = re.sub("([!?\"'])\\1+", r"\1", item)
    item = re.sub("([\.])\\1+", r" \1\1\1 ", item)
    item = re.sub("([!?])", r" \1 ", item)
    item = re.sub("([^\w\.,\-'\"!?])", r" ", item)
    item = ' '.join(item.split())
    #print(item)
    return item    


def cleaner_text(text, emoji_del=True, links_del=True, url_del=True):
    #regex for url
    urlregex = re.compile(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*')
    #regex for emoji
    emojiregex = re.compile(r'([\U0001F601-\U0001F64F]|[\U0001F900-\U0001F9FF]|[\U0001F680-\U0001F6FF])')
    #regx for links
    linksregex = re.compile(r'@\w+')
    #regex for hashtags
    hashregex = re.compile(r'#\w+')
    
    if(url_del):
        text = urlregex.sub('',text)
    if(links_del):
        text = linksregex.sub('',text)
    if(emoji_del):
        text = emojiregex.sub('',text)
    text = hashregex.sub('',text)
    
    text = filter(text)
    return text

In [3]:
#consumer key, consumer secret, access token, access secret.
ckey=""
csecret=""
atoken=""
asecret=""
fmt = '%Y-%m-%d %H:%M:%S'
# connection to db
conn = sqlite3.connect('C:/Users/Maxim/Nero/Chinook_Sqlite.sqlite')
c = conn.cursor()

In [4]:
class listener(StreamListener):

    def on_data(self, data):
        all_data = json.loads(data)
        tweet = all_data["text"]
        dt = all_data["created_at"]
        id = all_data["id"]
        #followers = all_data["followers"]
        cl_tweel = cleaner_text(tweet)
        user_id = all_data["user"]["id"]
        fl = all_data["user"]["followers_count"]
        #print((id,0,cl_tweel,dt,user_id,fl))

        if (not(cl_tweel == '')):
            c.execute("insert into rtwi (id_tweet,sentiment,review,datatme,us_id,followers)values (?,?,?,?,?,?)",
                   (id,0,cl_tweel,dt,user_id,fl))
        conn.commit()

        return True


    def on_error(self, status):
        print(status)

In [ ]:
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["bitcoin"])

In [ ]:
conn.close()